In [18]:
import numpy as np
from scipy.stats import norm
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 加载鸢尾花数据集
iris = load_iris()

# 将数据集划分为训练集和验证集
X, y = iris.data, iris.target
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义优化问题和目标函数
def objective_function(params):
    C, gamma = params
    svm = SVC(C=C, gamma=gamma)
    svm.fit(X_train, y_train)
    score = svm.score(X_val, y_val)
    return -score  # 最大化准确率

# 初始化
bounds = [(0.1, 10.0), (0.01, 1.0)]  # 参数的搜索范围
n_candidates = 10  # 每次迭代选择的候选点数量
n_iterations = 10  # 最大迭代次数
k = 5  # 选择EI值最大的k个点作为新的候选点

# EGO算法
initial_point = [np.random.uniform(bounds[0][0], bounds[0][1]), np.random.uniform(bounds[1][0], bounds[1][1])]
X = np.array([initial_point])
y = np.array([objective_function(initial_point)])


for _ in range(n_iterations):
    # 构建代理模型1
    kernel1 = ConstantKernel(1.0) * RBF(1.0)
    gpr1 = GaussianProcessRegressor(kernel=kernel1)
    gpr1.fit(X, y)

    # 构建代理模型2
    kernel2 = ConstantKernel(1.0) * RBF(1.0)
    gpr2 = GaussianProcessRegressor(kernel=kernel2)
    gpr2.fit(X, y)

    # 计算期望改进
    x_candidates = np.random.uniform(bounds[0][0], bounds[0][1], size=(n_candidates,))
    y_candidates = np.random.uniform(bounds[1][0], bounds[1][1], size=(n_candidates,))
    X_candidates = np.vstack((x_candidates, y_candidates)).T
    y_best = np.min(y)

    mu1, sigma1 = gpr1.predict(X_candidates, return_std=True)
    improvement1 = y_best - mu1
    ei1 = improvement1 * norm.cdf(improvement1 / sigma1) + sigma1 * norm.pdf(improvement1 / sigma1)

    mu2, sigma2 = gpr2.predict(X_candidates, return_std=True)
    improvement2 = y_best - mu2
    ei2 = improvement2 * norm.cdf(improvement2 / sigma2) + sigma2 * norm.pdf(improvement2 / sigma2)

    # 综合两个代理模型的EI值
    ei_combined = (ei1 + ei2) / 2

    # 选择EI值最大的k个点作为新的候选点
    selected_indices = np.argsort(ei_combined)[-k:]
    selected_candidates = X_candidates[selected_indices]

    # 在选定的候选点上进行实际试验
    y_selected = np.array([objective_function(candidate) for candidate in selected_candidates])

    # 更新代理模型
    X = np.vstack((X, selected_candidates))
    y = np.append(y, y_selected)

# 找到最优解
best_index = np.argmin(y)
best_params = X[best_index]

print("最优参数：", best_params)


最优参数： [9.91451749 0.6249183 ]


c:\Users\user\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:430:

In [20]:
C, gamma = best_params
svm = SVC(C=C, gamma=gamma)
svm.fit(X_train, y_train)
score = svm.score(X_val, y_val)

1.0
